In [23]:
library(ggplot2)
library(GGally)
library(lme4)

Loading required package: Matrix


In [11]:
path <- '/nfs/turbo/intmed-bnallamo-turbo/wsliu/Data/NRD/'

In [12]:
dat <- read.csv(file = paste0(path, 'cohorts/ami/patients_first.csv'))

In [17]:
comorbs <- c('CM_AIDS', 'CM_ALCOHOL', 'CM_ANEMDEF', 'CM_ARTH', 'CM_BLDLOSS', 'CM_CHF', 'CM_CHRNLUNG', 'CM_COAG', 'CM_DEPRESS', 'CM_DM', 'CM_DMCX', 'CM_DRUG', 'CM_HTN_C', 'CM_HYPOTHY', 'CM_LIVER', 'CM_LYMPH', 'CM_LYTES', 'CM_METS', 'CM_NEURO', 'CM_OBESE', 'CM_PARA', 'CM_PERIVASC', 'CM_PSYCH', 'CM_PULMCIRC', 'CM_RENLFAIL', 'CM_TUMOR', 'CM_ULCER', 'CM_VALVE', 'CM_WGHTLOSS')

In [18]:
dat$HOSP_NRD <- as.factor(dat$HOSP_NRD)
dat$FEMALE <- as.factor(dat$FEMALE)
for(c in comorbs){
    dat[ , c] <- as.factor(dat[ , c])
}

dat$readm30 <- as.numeric(dat$readm30) - 1

In [45]:
head(dat)

NRD_VisitLink,CM_AIDS,CM_ALCOHOL,CM_ANEMDEF,CM_ARTH,CM_BLDLOSS,CM_CHF,CM_CHRNLUNG,CM_COAG,CM_DEPRESS,⋯,CM_TUMOR,CM_ULCER,CM_VALVE,CM_WGHTLOSS,KEY_NRD,AGE,FEMALE,HOSP_NRD,NRD_DaysToEvent,readm30
b000fk0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,10576754,65,0,12929,20821,0
b000vym,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,19785198,72,0,12175,18702,0
b0015un,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,19212175,57,1,12024,17420,0
b0016wx,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,21012648,61,0,12307,18289,0
b001gse,0,0,0,0,0,1,1,0,0,⋯,0,0,0,1,18432816,61,0,12245,22681,1
b001n49,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,16083400,49,0,12298,12832,0


In [15]:
colnames(dat)

[1] "NRD_VisitLink"   "CM_AIDS"         "CM_ALCOHOL"      "CM_ANEMDEF"     
 [5] "CM_ARTH"         "CM_BLDLOSS"      "CM_CHF"          "CM_CHRNLUNG"    
 [9] "CM_COAG"         "CM_DEPRESS"      "CM_DM"           "CM_DMCX"        
[13] "CM_DRUG"         "CM_HTN_C"        "CM_HYPOTHY"      "CM_LIVER"       
[17] "CM_LYMPH"        "CM_LYTES"        "CM_METS"         "CM_NEURO"       
[21] "CM_OBESE"        "CM_PARA"         "CM_PERIVASC"     "CM_PSYCH"       
[25] "CM_PULMCIRC"     "CM_RENLFAIL"     "CM_TUMOR"        "CM_ULCER"       
[29] "CM_VALVE"        "CM_WGHTLOSS"     "KEY_NRD"         "AGE"            
[33] "FEMALE"          "HOSP_NRD"        "NRD_DaysToEvent" "readm30"

In [46]:
result <- glmer(readm30 ~ CM_AIDS + CM_ALCOHOL + CM_ANEMDEF + CM_ARTH + CM_BLDLOSS + CM_CHF + CM_CHRNLUNG + CM_COAG + 
                CM_DEPRESS + CM_DM + CM_DMCX + CM_DRUG + CM_HTN_C + CM_HYPOTHY + CM_LIVER + CM_LYMPH + CM_LYTES + CM_METS + 
                CM_NEURO + CM_OBESE + CM_PARA + CM_PERIVASC + CM_PSYCH + CM_PULMCIRC + CM_RENLFAIL + CM_TUMOR + CM_ULCER + 
                CM_VALVE + CM_WGHTLOSS + AGE + FEMALE + (1|HOSP_NRD), data = dat, family = binomial, 
                control = glmerControl(optimizer = "nloptwrap", calc.derivs = FALSE), nAGQ = 0)

In [30]:
print(result, corr=False)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: readm30 ~ CM_AIDS + CM_ALCOHOL + CM_ANEMDEF + CM_ARTH + CM_BLDLOSS +  
    CM_CHF + CM_CHRNLUNG + CM_COAG + CM_DEPRESS + CM_DM + CM_DMCX +  
    CM_DRUG + CM_HTN_C + CM_HYPOTHY + CM_LIVER + CM_LYMPH + CM_LYTES +  
    CM_METS + CM_NEURO + CM_OBESE + CM_PARA + CM_PERIVASC + CM_PSYCH +  
    CM_PULMCIRC + CM_RENLFAIL + CM_TUMOR + CM_ULCER + CM_VALVE +  
    CM_WGHTLOSS + AGE + FEMALE + (1 | HOSP_NRD)
   Data: dat
      AIC       BIC    logLik  deviance  df.resid 
142782.76 143118.18 -71358.38 142716.76    191755 
Random effects:
 Groups   Name        Std.Dev.
 HOSP_NRD (Intercept) 0.1922  
Number of obs: 191788, groups:  HOSP_NRD, 898
Fixed Effects:
 (Intercept)      CM_AIDS1   CM_ALCOHOL1   CM_ANEMDEF1      CM_ARTH1  
    -3.42750       0.08255       0.03555       0.27625       0.15406  
 CM_BLDLOSS1       CM_CHF1  CM_CHRNLUNG1      CM_COAG1   CM_DEPRESS1  

In [48]:
test <- dat[1:1000, ]

In [51]:
pred_score <- predict(result, newdata = test, type = 'response')

In [52]:
cbind(pred_score[1:100], test$readm30[1:100])

1,0.14421975,0
2,0.11434421,0
3,0.06991293,0
4,0.06515673,0
5,0.50817250,1
6,0.04964932,0
7,0.12926063,0
8,0.17981672,0
9,0.09816497,0
10,0.14272805,0
11,0.33726592,0
